# Relative FFT Pose Analysis

This notebook analyzes the relative FFT pose data and prepares it for comparison with Sonar and DVL measurements.

The FFT data contains:
- **time**: timestamp in Unix seconds
- **distance**: distance measurement in centimeters  
- **heading**: heading in radians
- **pitch**: pitch angle in radians

## 1. Import Libraries and Load Data

Import necessary libraries and load the relative FFT pose data.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path

# Import our custom FFT analysis utilities
from utils.relative_fft_analysis import (
    load_relative_fft_data,
    analyze_fft_data_quality,
    convert_fft_to_xy_coordinates,
    create_three_system_comparison_plot
)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Configuration
FFT_CSV_PATH = Path("/Volumes/LaCie/SOLAQUA/relative_fft_pose/2024-08-20_17-02-00_relative_pose_fft.csv")

print(f"FFT CSV Path: {FFT_CSV_PATH}")
print(f"File exists: {FFT_CSV_PATH.exists()}")

# Load the relative FFT data
if FFT_CSV_PATH.exists():
    df_fft = load_relative_fft_data(FFT_CSV_PATH)
    print(f"✓ Loaded FFT data: {len(df_fft)} records")
    print(f"Time range: {df_fft['timestamp'].min()} to {df_fft['timestamp'].max()}")
    
    # Display sample data
    print("\nSample FFT data:")
    print(df_fft.head())
else:
    print(f"✗ FFT data file not found: {FFT_CSV_PATH}")
    df_fft = None

FFT CSV Path: /Volumes/LaCie/SOLAQUA/relative_fft_pose/2024-08-20_17-02-00_relative_pose_fft.csv
File exists: True
✓ Loaded 410 relative FFT pose records
✓ Loaded FFT data: 410 records
Time range: 2024-08-20 15:02:03.753343105 to 2024-08-20 15:03:00.519725800

Sample FFT data:
           time  distance   heading     pitch                     timestamp  \
0  1.724166e+09  0.000323 -1.189418  0.750176 2024-08-20 15:02:03.753343105   
1  1.724166e+09  0.000087 -1.173581  0.586265 2024-08-20 15:02:04.286877632   
2  1.724166e+09  0.024386  1.250215  1.438593 2024-08-20 15:02:04.366403103   
3  1.724166e+09  0.093198  1.461399  1.428290 2024-08-20 15:02:04.683693647   
4  1.724166e+09  0.000892  1.270715  0.817889 2024-08-20 15:02:04.915297031   

     distance_m  pitch_rad  pitch_deg  heading_rad  heading_deg  
0  3.230434e-06   0.750176  42.981929    -1.189418   -68.148656  
1  8.718776e-07   0.586265  33.590534    -1.173581   -67.241221  
2  2.438620e-04   1.438593  82.425327     1.25021

## 2. Analyze Data Quality

Analyze the quality and characteristics of the relative FFT data.

In [3]:
if df_fft is not None:
    # Analyze data quality
    fft_stats = analyze_fft_data_quality(df_fft)
    
    print("=== RELATIVE FFT DATA QUALITY ANALYSIS ===")
    print(f"Total measurements: {fft_stats['total_measurements']}")
    print(f"Time range: {fft_stats['time_range_seconds']:.1f} seconds")
    print(f"Sampling rate: {fft_stats['sampling_rate_hz']:.2f} Hz")
    
    print(f"\nDistance measurements:")
    print(f"  Count: {fft_stats['distance_count']}")
    print(f"  Range: {fft_stats['distance_min_m']:.3f} to {fft_stats['distance_max_m']:.3f} m")
    print(f"  Mean: {fft_stats['distance_mean_m']:.3f} ± {fft_stats['distance_std_m']:.3f} m")
    
    print(f"\nPitch measurements:")
    print(f"  Count: {fft_stats['pitch_count']}")
    print(f"  Range: {fft_stats['pitch_min_deg']:.1f}° to {fft_stats['pitch_max_deg']:.1f}°")
    print(f"  Mean: {fft_stats['pitch_mean_deg']:.1f}° ± {fft_stats['pitch_std_deg']:.1f}°")
    
    print(f"\nHeading measurements:")
    print(f"  Count: {fft_stats['heading_count']}")
    print(f"  Range: {fft_stats['heading_min_deg']:.1f}° to {fft_stats['heading_max_deg']:.1f}°")
    print(f"  Mean: {fft_stats['heading_mean_deg']:.1f}° ± {fft_stats['heading_std_deg']:.1f}°")

=== RELATIVE FFT DATA QUALITY ANALYSIS ===
Total measurements: 410
Time range: 56.8 seconds
Sampling rate: 7.22 Hz

Distance measurements:
  Count: 410
  Range: -1.833 to 1.596 m
  Mean: 0.877 ± 0.320 m

Pitch measurements:
  Count: 410
  Range: -82.7° to 88.1°
  Mean: 3.8° ± 14.6°

Heading measurements:
  Count: 410
  Range: -84.0° to 83.7°
  Mean: 3.7° ± 14.3°


## 3. Convert to XY Coordinates

Convert the distance and pitch measurements to XY coordinates for comparison with other systems.

In [4]:
if df_fft is not None:
    # Convert to XY coordinates
    df_fft = convert_fft_to_xy_coordinates(df_fft)
    
    print("✓ Converted FFT measurements to XY coordinates")
    print(f"X range: {df_fft['fft_x'].min():.3f} to {df_fft['fft_x'].max():.3f} m")
    print(f"Y range: {df_fft['fft_y'].min():.3f} to {df_fft['fft_y'].max():.3f} m")
    
    # Display sample with XY coordinates
    print("\nSample FFT data with XY coordinates:")
    display_cols = ['timestamp', 'distance_m', 'pitch_deg', 'fft_x', 'fft_y']
    print(df_fft[display_cols].head())

✓ Converted FFT measurements to XY coordinates
X range: -1.821 to 1.573 m
Y range: -0.596 to 0.441 m

Sample FFT data with XY coordinates:
                      timestamp    distance_m  pitch_deg         fft_x  \
0 2024-08-20 15:02:03.753343105  3.230434e-06  42.981929  2.363285e-06   
1 2024-08-20 15:02:04.286877632  8.718776e-07  33.590534  7.262850e-07   
2 2024-08-20 15:02:04.366403103  2.438620e-04  82.425327  3.214546e-05   
3 2024-08-20 15:02:04.683693647  9.319766e-04  81.835004  1.323632e-04   
4 2024-08-20 15:02:04.915297031  8.924165e-06  46.861574  6.102016e-06   

          fft_y  
0  2.202406e-06  
1  4.823697e-07  
2  2.417340e-04  
3  9.225293e-04  
4  6.511998e-06  


## 4. Visualize FFT Data

Create plots to visualize the FFT measurements over time and in XY space.

In [5]:
if df_fft is not None:
    # Create visualization
    from plotly.subplots import make_subplots
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Distance Over Time',
            'XY Position Plot',
            'Pitch Over Time', 
            'Heading Over Time'
        ]
    )
    
    # Distance over time
    fig.add_trace(
        go.Scatter(x=df_fft['timestamp'], y=df_fft['distance_m'],
                  mode='lines+markers', name='Distance',
                  line=dict(color='blue'), marker=dict(size=4)),
        row=1, col=1
    )
    
    # XY position plot
    fig.add_trace(
        go.Scatter(x=df_fft['fft_x'], y=df_fft['fft_y'],
                  mode='markers', name='XY Position',
                  marker=dict(color='red', size=6)),
        row=1, col=2
    )
    
    # Pitch over time
    fig.add_trace(
        go.Scatter(x=df_fft['timestamp'], y=df_fft['pitch_deg'],
                  mode='lines+markers', name='Pitch',
                  line=dict(color='green'), marker=dict(size=4)),
        row=2, col=1
    )
    
    # Heading over time
    fig.add_trace(
        go.Scatter(x=df_fft['timestamp'], y=df_fft['heading_deg'],
                  mode='lines+markers', name='Heading',
                  line=dict(color='orange'), marker=dict(size=4)),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        title='Relative FFT Pose Data Analysis',
        height=800,
        showlegend=False
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Time", row=1, col=1)
    fig.update_yaxes(title_text="Distance (m)", row=1, col=1)
    
    fig.update_xaxes(title_text="X Position (m)", row=1, col=2)
    fig.update_yaxes(title_text="Y Position (m)", row=1, col=2)
    
    fig.update_xaxes(title_text="Time", row=2, col=1)
    fig.update_yaxes(title_text="Pitch (degrees)", row=2, col=1)
    
    fig.update_xaxes(title_text="Time", row=2, col=2)
    fig.update_yaxes(title_text="Heading (degrees)", row=2, col=2)
    
    fig.show()
    
    print("✓ FFT data visualization complete")

✓ FFT data visualization complete
